<a href="https://colab.research.google.com/github/guthi1/deep-learning/blob/main/generative_model/Finetune_Stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U autotrain-advanced -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 100.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 71.0 MB/s et

In [ ]:
!autotrain setup --update-torch

> INFO    Installing latest transformers@main
> INFO    Successfully installed latest transformers
> INFO    Installing latest peft@main
> INFO    Successfully installed latest peft
> INFO    Installing latest diffusers@main
> INFO    Successfully installed latest diffusers
> INFO    Installing latest trl@main
> INFO    Successfully installed latest trl
> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [ ]:
import os


if not os.path.exists("images"): ##  you need to put 7-10 your images
  os.mkdir("images")
if not os.path.exists("images_refined"): ## new generated image will save here
  os.mkdir("images_refined")
if not os.path.exists("model"): ## Lora model will save
  os.mkdir("model")

In [ ]:
!autotrain dreambooth \
--model stabilityai/stable-diffusion-xl-base-1.0 \
--output model/ \
--image-path images/ \
--prompt "photo of Guillaume Thibault" \
--resolution 512 \
--batch-size 1 \
--num-steps 500 \
--fp16 \
--gradient-accumulation 4 \
--lr 1e-4 \
--use-8bit-adam


In [63]:
from diffusers import DiffusionPipeline, StableDiffusionXLImg2ImgPipeline
import torch
import random

model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.bfloat16,
)
pipe.to("cuda")
pipe.load_lora_weights("model/", weight_name="pytorch_lora_weights.safetensors")

refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.bfloat16,
)
refiner.to("cuda")


prompt = "a portrait of Guillaume Thibault, pixar, cartoon, 3d, headshots, 4k, uhd"
# prompt = "a portrait of Guillaume Thibault, 80s anime, headshots, retro, 4k, uhd"
# prompt = "a portrait of Guillaume Thibault in the style of rembrandt, headshots, 4k, uhd"
# prompt = "a portrait of Guillaume Thibault Cowboy Bebop or Trigun:: space western theme:: western sci-fi --niji"
# prompt = "an image of a Guillaume Thibault near a neon sign, in the style of cyberpunk manga, ruslan lobanov, detailed costumes, 32k uhd, zeng chuangxing, daniel f. gerhartz, calculated --ar 77:100 --s 750 --niji 5"
# prompt = "(masterpiece of the highest quality:1. 4), headshots of Guillaume Thibault, perfect blue depicted with artistic brilliance (by junji ito and dave mckean and bridget riley), cel animation, vivid colors, 16K, UHD, HDR, (Masterpiece, best quality:1. 5)"
# prompt = "Guillaume Thibault manga legend of anime vh1, in the style of miho hirano, dynamic and action-packed scenes, gaston bussière, chinese new year festivities, ross tran, charming character illustrations, light red and bronze --ar 73:88 --s 750 --niji 5"
# prompt = "Internet artwork Masterpiece, Best Quality, Guillaume Thibault, 8k, UHD, HDR . Browser as venue, leveraging networked interconnectivity as artistic material, natively digital art in the expanded space of hypertext"
prompt = "Anime artwork, Guillaume Thibault studying at a school desk. Darkened eyes, anime art style, dorm scene, . anime style, key visual, vibrant, studio anime, highly detailed."

for seed in range(4):
    seed = random.randint(1,100)
    generator = torch.Generator("cuda").manual_seed(seed)
    image = pipe(prompt=prompt, generator=generator, num_inference_steps=25)
    image = image.images[0]
    image = refiner(prompt=prompt, generator=generator, image=image)
    image = image.images[0]
    image.save(f"images_refined/{prompt}_{seed}.png")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]